## Part-4：Chain
### Part-4.1：Chain -> LLMChain
**LLMChain最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用**

In [ ]:
import os
import openai
import configparser
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import DatetimeOutputParser
from langchain import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain


conf = configparser.ConfigParser()
current_directory = os.path.dirname(os.path.realpath('__file__'))
config_file_path = os.path.join(current_directory, '..', '..', '..', 'config.ini')
conf.read(config_file_path)
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [ ]:

llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"],
    template=prompt_template
)
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# print(chain.run({
#     'adjective': "dogs jokes"
# }))

### Part-4.2：Chain -> SimpleSequentialChain
**这是一个SimpleSequentialChain，按顺序运行这两个链，简单形式，单一输入/输出，并且一个步骤的输出是下一个步骤的输入。**

In [ ]:

output_parser = CommaSeparatedListOutputParser()

prompt_1 = PromptTemplate(template="你是一个游戏原画设计师，根据关键词生成描述信息，关键词：{keywords}",
                          input_variables=["keywords"])

prompt_2 = PromptTemplate(template="根据的文本内容，生成原画关键词列表。\n文本内容：{text_content} \n{output_prompt}",
                          input_variables=["text_content"],
                          partial_variables={"output_prompt": output_parser.get_format_instructions()})

chain1 = LLMChain(llm=llm, prompt=prompt_1)
chain2 = LLMChain(llm=llm, prompt=prompt_2)
simple_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

In [ ]:
result_art_designer = simple_chain.run("慈祥的白发苍苍的老奶奶")
output_parser.parse(result_art_designer)

### Part-4.3：Chain -> SequentialChain
**SequentialChain 更通用形式的顺序链，允许多个输入/输出。**
#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

In [ ]:

llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

prompt_1 = PromptTemplate(template="你是一个游戏原画设计师，根据关键词生成描述信息，关键词：{keywords}",
                          input_variables=["keywords"])

prompt_2 = PromptTemplate(template="根据的文本内容，生成原画关键词列表。\n文本内容：{text_content} \n{output_prompt}",
                          input_variables=["text_content"],
                          partial_variables={"output_prompt": output_parser.get_format_instructions()})

chain3 = LLMChain(llm=llm, prompt=prompt_1, output_key="text_content")
chain4 = LLMChain(llm=llm, prompt=prompt_2, output_key="keywords_list")
se_chain = SequentialChain(chains=[chain3, chain4], input_variables=["keywords"],
                           output_variables=["text_content", "keywords_list"], verbose=True)

In [ ]:
result_se_chain = se_chain({"keywords": "12岁的npc少年"})

In [ ]:
output_parser.parse(result_se_chain.get("text_content"))